### <p style="text-align: right;"> &#9989; Andrew Koren.</p>

# PHY480 Day 19

## In-class assignment: Metropolis algorithm

In this in-class assignment we experiment with the Metropolis algorithm.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


**Task 1.** Consider the following probability density function (PDF):

$$
P(x)\sim (1-x^2)^{3/2},\,\,\,\,\,x\in[-1,1].
$$

Find the proper normalization so that the integral of the PDF is 1. Compute the mean and the variance. This will help the comparisons later.


In [13]:
import sympy as sp

x = sp.symbols('x')
sp.integrate((1-(x**2))**(3/2), (x, -1, 1)).evalf()

1.17809724509617

Mean $\mu = \int_a^b x f(x) dx$ <br>
Variance $\int_a^b x^2 f(x) dx - \mu^2$

Looks like we have
$P(x) = \frac{\left(1-x^2\right)^{3/2}}{1.178}$

**Task 2.** Code the Metropolis algorithm where the proposed moves are drawn from the uniform distribution, $x\in[-1,1]$.


Notes: 

initialize x = x0 <br>
accpt = 0 <br>
for i = 1->N draw x' from Q(x) <br>
Q(x) = 1 &  x' = 2r-1 (arbitrary, q can be different)<br>

Compute $T_A = \min\left\{ 1, \frac{P(x')}{Q(x')}/\frac{P(x)}{Q(x)} \right\}$

Probability of staying vs going.



In [ ]:
# sample (1-x^2)^{3/2} distribution, the proposal distribution is uniform
# Input:
# x -- current value
# Output:
# x, n -- x is the new value (may be the same if the move was rejected),
#         n = 1, if the new move was accepted, and = 0, if the move was rejected
def get_rand_x32_flat( x ):
    
    # proposal
    
    # acceptance probability

    # ...

    return # something


**Task 3.** Generate a number of samples from the PDF you coded above, make a histogram and compare with the (normalized) PDF. Also, compute and print the acceptance rate.

In [ ]:
# random seed for reproducibility
np.random.seed(3)


In [ ]:
# YOUR CODE HERE


**Task 4.** Compute the mean and the variance from the data and compare with the exact results.


In [ ]:
# YOUR CODE HERE


**Task 5.** Compute the autocorrelation function:

$$
C(t)=\frac{1}{N-t_{max}}\sum_{i=1}^{N-t_{max}}(x_i-\bar x)(x_{i+t}-\bar x),
$$

where $\bar x$ is the mean. Use $t_{max}=10$. Plot the normalized autocorrelation function

$$
c(t)=\frac{C(t)}{C(0)}.
$$

For comparison, generate a series of random numbers uniformly distributed in $[0,1)$, compute and plot their autocorrelation function.


In [ ]:
# YOUR CODE HERE


**Task 6.** Modify the Metropolis algorithm so that the proposed moves are drawn from the PDF:

$$
Q(x)\sim \cos\left(\frac{\pi x}{2}\right).
$$

(Plot $P(x)$ and $Q(x)$ to understand why this proposal distribution may be more beneficial than the uniform one.)



In [ ]:
# sample (1-x^2)^{3/2} distribution, the proposal distribution is cos(pi*x/2)
# Input:
# x -- current value
# Output:
# x, n -- x is the new value (may be the same if the move was rejected),
#         n = 1, if the new move was accepted, and = 0, if the move was rejected
def get_rand_x32_cos( x ):

    # proposal
    
    # acceptance probability

    # ...

    return # something


**Task 7.** Repeat the previous tasks for the modified Metropolis algorithm: generate a sequence, plot a histogram, compute the mean and the variance, compute and plot the autocorrelation function. Compare the acceptance rate from the modified to the original version of the Metropolis algorithm with the uniform proposal distribution.


In [ ]:
# YOUR CODE HERE


&#169; Copyright 2025,  Michigan State University Board of Trustees